# Pagination

> Pagination pattern with automatic route generation and state management

In [ ]:
#| default_exp patterns.pagination

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional, Callable, List
from enum import Enum
from dataclasses import dataclass
from fasthtml.common import *
from fastcore.basics import patch

from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_sizes, btn_styles, btn_behaviors
from cjm_fasthtml_daisyui.components.navigation.pagination import join
from cjm_fasthtml_tailwind.utilities.spacing import m
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, justify
from cjm_fasthtml_tailwind.core.base import combine_classes

## Pagination Style Options

The `PaginationStyle` enum defines different pagination display styles.

In [ ]:
#| export
class PaginationStyle(Enum):
    """Display styles for pagination controls."""
    SIMPLE = "simple"        # Previous/Next buttons with page info  
    COMPACT = "compact"      # Previous/Next buttons only (no page info)

## Pagination Class

The `Pagination` class manages paginated views with automatic route generation and state management. It follows the same pattern as `StepFlow` for consistent API design.

Key features:
- Automatic pagination math (total pages, page ranges)
- Auto-generated routes via `create_router()`
- Query parameter preservation (filters, sorting, etc.)
- Flexible data loading and rendering
- HTMX integration for SPA-like navigation

In [ ]:
#| export
class Pagination:
    """Manage paginated views with automatic route generation and state management."""
    
    def __init__(
        self,
        pagination_id: str,  # Unique identifier for this pagination instance
        data_loader: Callable[[Any], List[Any]],  # Function that returns all items
        render_items: Callable[[List[Any], int, Any], Any],  # Function to render items for a page
        items_per_page: int = 20,  # Number of items per page
        container_id: str = None,  # HTML ID for container (auto-generated if None)
        content_id: str = None,  # HTML ID for content area (auto-generated if None)
        preserve_params: List[str] = None,  # Query parameters to preserve
        style: PaginationStyle = PaginationStyle.SIMPLE,  # Pagination display style
        prev_text: str = "« Previous",  # Text for previous button
        next_text: str = "Next »",  # Text for next button
        page_info_format: str = "Page {current} of {total}",  # Format for page info
        button_size: str = None,  # Button size class
        push_url: bool = True,  # Whether to update URL with hx-push-url
        show_endpoints: bool = False,  # Whether to show First/Last buttons
        first_text: str = "«« First",  # Text for first page button
        last_text: str = "Last »»",  # Text for last page button
    ):
        """Initialize pagination manager."""
        self.pagination_id = pagination_id
        self.data_loader = data_loader
        self.render_items = render_items
        self.items_per_page = items_per_page
        self.preserve_params = preserve_params or []
        self.style = style
        self.prev_text = prev_text
        self.next_text = next_text
        self.page_info_format = page_info_format
        self.button_size = button_size
        self.push_url = push_url
        self.show_endpoints = show_endpoints
        self.first_text = first_text
        self.last_text = last_text
        
        # Auto-generate IDs if not provided
        self.container_id = container_id or InteractionHtmlIds.pagination_container(pagination_id)
        self.content_id = content_id or InteractionHtmlIds.pagination_content(pagination_id)

## Helper Methods

In [ ]:
#| export
@patch
def get_total_pages(self:Pagination, 
                    total_items: int  # Total number of items
                   ) -> int:  # Total number of pages
    """Calculate total number of pages."""
    return max(1, (total_items + self.items_per_page - 1) // self.items_per_page)

In [ ]:
#| export
@patch
def get_page_items(self:Pagination,
                   all_items: List[Any],  # All items
                   page: int  # Current page number (1-indexed)
                  ) -> tuple:  # (page_items, start_idx, end_idx)
    """Get items for the current page."""
    start_idx = (page - 1) * self.items_per_page
    end_idx = min(start_idx + self.items_per_page, len(all_items))
    page_items = all_items[start_idx:end_idx]
    return page_items, start_idx, end_idx

In [ ]:
#| export
@patch
def build_route(self:Pagination,
                page: int,  # Page number
                request: Any,  # FastHTML request object
                page_route_func: Callable  # Route function from create_router
               ) -> str:  # Complete route with preserved params
    """Build route URL with preserved query parameters."""
    # Get preserved params from request
    params = {"page": page}
    for param in self.preserve_params:
        value = request.query_params.get(param)
        if value:
            params[param] = value
    
    # Build route with params
    return page_route_func.to(**params)

## Rendering Methods

In [ ]:
#| export
@patch
def render_navigation_controls(self:Pagination,
                               current_page: int,  # Current page number
                               total_pages: int,  # Total number of pages
                               route_func: Callable[[int], str]  # Function to generate route for page
                              ) -> FT:  # Navigation controls element
    """Render pagination navigation controls."""
    # Calculate prev/next page numbers
    prev_page = max(1, current_page - 1)
    next_page = min(total_pages, current_page + 1)
    
    # Check if we're at boundaries
    is_first_page = current_page <= 1
    is_last_page = current_page >= total_pages
    
    # Build button classes
    button_classes = [btn]
    if self.button_size:
        button_classes.append(self.button_size)
    
    # Build button group content
    button_group_parts = []
    
    # Add First button if enabled
    if self.show_endpoints:
        first_button = A(
            self.first_text,
            href=route_func(1),
            hx_get=route_func(1),
            hx_target=InteractionHtmlIds.as_selector(self.content_id),
            hx_swap="outerHTML",
            hx_push_url="true" if self.push_url else None,
            cls=combine_classes(
                *button_classes,
                btn_styles.ghost if not is_first_page else btn_behaviors.disabled
            ),
            disabled=is_first_page
        )
        button_group_parts.append(first_button)
    
    # Create Previous button
    prev_button = A(
        self.prev_text,
        href=route_func(prev_page),
        hx_get=route_func(prev_page),
        hx_target=InteractionHtmlIds.as_selector(self.content_id),
        hx_swap="outerHTML",
        hx_push_url="true" if self.push_url else None,
        cls=combine_classes(
            *button_classes,
            btn_styles.ghost if not is_first_page else btn_behaviors.disabled
        ),
        disabled=is_first_page
    )
    button_group_parts.append(prev_button)
    
    # Add page info for SIMPLE style
    if self.style == PaginationStyle.SIMPLE:
        page_info = self.page_info_format.format(
            current=current_page,
            total=total_pages
        )
        button_group_parts.append(
            Span(page_info, cls=str(m.x(4)))
        )
    
    # Create Next button
    next_button = A(
        self.next_text,
        href=route_func(next_page),
        hx_get=route_func(next_page),
        hx_target=InteractionHtmlIds.as_selector(self.content_id),
        hx_swap="outerHTML",
        hx_push_url="true" if self.push_url else None,
        cls=combine_classes(
            *button_classes,
            btn_styles.ghost if not is_last_page else btn_behaviors.disabled
        ),
        disabled=is_last_page
    )
    button_group_parts.append(next_button)
    
    # Add Last button if enabled
    if self.show_endpoints:
        last_button = A(
            self.last_text,
            href=route_func(total_pages),
            hx_get=route_func(total_pages),
            hx_target=InteractionHtmlIds.as_selector(self.content_id),
            hx_swap="outerHTML",
            hx_push_url="true" if self.push_url else None,
            cls=combine_classes(
                *button_classes,
                btn_styles.ghost if not is_last_page else btn_behaviors.disabled
            ),
            disabled=is_last_page
        )
        button_group_parts.append(last_button)
    
    # Build container classes
    container_classes = [
        flex_display,
        justify.center,
        m.t(6)
    ]
    
    # Create the pagination container
    return Div(
        Div(
            *button_group_parts,
            cls=str(join)
        ),
        cls=combine_classes(*container_classes)
    )

In [ ]:
#| export
@patch
def render_page_content(self:Pagination,
                       page_items: List[Any],  # Items for current page
                       current_page: int,  # Current page number
                       total_pages: int,  # Total number of pages
                       request: Any,  # FastHTML request object
                       route_func: Callable[[int], str]  # Function to generate route for page
                      ) -> FT:  # Complete page content with items and navigation
    """Render complete page content with items and pagination controls."""
    # Render items using provided render function
    items_content = self.render_items(page_items, current_page, request)
    
    # Render navigation controls
    nav_controls = self.render_navigation_controls(current_page, total_pages, route_func)
    
    # Combine items and navigation
    return Div(
        items_content,
        nav_controls if total_pages > 1 else None,
        id=self.content_id
    )

## Route Generation

In [ ]:
#| export
@patch
def create_router(self:Pagination,
                  prefix: str = ""  # URL prefix for routes (e.g., "/library")
                 ) -> APIRouter:  # APIRouter with generated routes
    """Create FastHTML router with generated routes for pagination."""
    router = APIRouter(prefix=prefix)
    
    # Store reference to pagination in router for access in route handlers
    router.pagination = self
    
    # Main content route
    @router
    def content(request, page: int = 1, **kwargs):
        """Handle paginated content requests."""
        # Load all items
        all_items = self.data_loader(request)
        
        # Calculate pagination
        total_pages = self.get_total_pages(len(all_items))
        page_items, start_idx, end_idx = self.get_page_items(all_items, page)
        
        # Build route function for navigation
        def make_route(p: int) -> str:
            return self.build_route(p, request, content)
        
        # Render page content
        return self.render_page_content(
            page_items=page_items,
            current_page=page,
            total_pages=total_pages,
            request=request,
            route_func=make_route
        )
    
    return router

## Usage Example

Here's a complete example showing how to create a paginated view:

In [ ]:
# Example: Simple paginated item list
from fasthtml.common import Div, H3, P, Li, Ul

# Define data loader function
def load_items(request):
    """Load all items - this could query a database, file system, etc."""
    # For demo, generate 100 items
    return [f"Item {i}" for i in range(1, 101)]

# Define render function for items
def render_items_list(items, page, request):
    """Render items for current page."""
    return Ul(
        *[Li(item) for item in items],
        cls="list-disc ml-6"
    )

# Create pagination instance
items_pagination = Pagination(
    pagination_id="items",
    data_loader=load_items,
    render_items=render_items_list,
    items_per_page=10
)

# Generate router
items_router = items_pagination.create_router(prefix="/items")

# In your FastHTML app, register the router:
# from cjm_fasthtml_app_core.core.routing import register_routes
# register_routes(app, items_router)
#
# Or directly:
# items_router.to_app(app)

# Access the pagination at: /items/content?page=1

# Test pagination math
print(f"Items per page: {items_pagination.items_per_page}")
print(f"Total pages for 100 items: {items_pagination.get_total_pages(100)}")
print(f"Total pages for 95 items: {items_pagination.get_total_pages(95)}")

# Test page items
all_items = load_items(None)
page1_items, start, end = items_pagination.get_page_items(all_items, 1)
print(f"\nPage 1: items {start+1}-{end} = {page1_items[:3]}...")

page5_items, start, end = items_pagination.get_page_items(all_items, 5)
print(f"Page 5: items {start+1}-{end} = {page5_items[:3]}...")

Items per page: 10
Total pages for 100 items: 10
Total pages for 95 items: 10

Page 1: items 1-10 = ['Item 1', 'Item 2', 'Item 3']...
Page 5: items 41-50 = ['Item 41', 'Item 42', 'Item 43']...


In [ ]:
# Example 2: Pagination with preserved query parameters (filters, search, etc.)
from fasthtml.common import Div, H3, P

# Define data loader that uses query parameters
def load_media_files(request):
    """Load media files, filtered by query parameters."""
    # In real app, this would filter based on request.query_params
    media_type = request.query_params.get("media_type", "all") if request else "all"
    view = request.query_params.get("view", "grid") if request else "grid"
    
    # Simulate filtered results
    all_files = [f"file{i}.mp4" for i in range(1, 51)]
    
    # Return filtered files (simplified for demo)
    return all_files

# Define render function that uses query parameters
def render_media_grid(items, page, request):
    """Render media items in grid view."""
    media_type = request.query_params.get("media_type", "all") if request else "all"
    view = request.query_params.get("view", "grid") if request else "grid"
    
    return Div(
        H3(f"Media Library ({view} view, filter: {media_type})"),
        P(f"Showing {len(items)} files on page {page}"),
        Div(*[Div(item) for item in items])
    )

# Create pagination with preserved parameters
media_pagination = Pagination(
    pagination_id="media",
    data_loader=load_media_files,
    render_items=render_media_grid,
    items_per_page=20,
    preserve_params=["view", "media_type"],  # Automatically preserve these in URLs
    style=PaginationStyle.SIMPLE
)

# Generate router
media_router = media_pagination.create_router(prefix="/library")

# Now when users navigate pages, view and media_type are automatically preserved:
# /library/content?page=1&view=grid&media_type=video
# /library/content?page=2&view=grid&media_type=video  <- view and media_type preserved!

print("Media pagination configured with parameter preservation:")
print(f"  Preserved params: {media_pagination.preserve_params}")
print(f"  Items per page: {media_pagination.items_per_page}")
print(f"  Style: {media_pagination.style.value}")

Media pagination configured with parameter preservation:
  Preserved params: ['view', 'media_type']
  Items per page: 20
  Style: simple


## Integration with FastHTML Applications

The `Pagination` class integrates seamlessly with FastHTML applications using the same pattern as `StepFlow`.

In [ ]:
# This cell has been replaced by the new Pagination class above

## Router Registration

After creating a `Pagination` instance and generating its router, register it with your FastHTML app:

```python
from cjm_fasthtml_app_core.core.routing import register_routes

# Create pagination
library_pagination = Pagination(
    pagination_id="library",
    data_loader=load_library_items,
    render_items=render_library_grid,
    items_per_page=20,
    preserve_params=["view", "media_type"]
)

# Generate router
library_router = library_pagination.create_router(prefix="/library")

# Register with app
register_routes(app, library_router)

# Or directly:
# library_router.to_app(app)
```

The router automatically handles:
- Pagination math (total pages, item slicing)
- Query parameter preservation
- Route generation with preserved state
- HTMX integration

## Data Loader and Render Functions

The `Pagination` class requires two key functions:

### Data Loader Function

Loads all items based on request (can use query parameters for filtering):

```python
def load_items(request):
    """Load all items - can filter based on query parameters."""
    # Access filters from query parameters
    media_type = request.query_params.get("media_type", "all")
    search = request.query_params.get("search", "")
    
    # Load and filter items
    items = get_all_items()
    
    if media_type != "all":
        items = [item for item in items if item.type == media_type]
    
    if search:
        items = [item for item in items if search.lower() in item.name.lower()]
    
    return items
```

### Render Function

Renders the items for display (receives current page items, page number, and request):

```python
def render_items(items, page, request):
    """Render items for current page."""
    # Can access query parameters if needed
    view_mode = request.query_params.get("view", "grid")
    
    if view_mode == "grid":
        return render_grid(items)
    else:
        return render_list(items)
```

The pagination automatically:
- Calls `data_loader` to get all items
- Slices items for current page
- Passes page items to `render_items`
- Adds navigation controls
- Preserves specified query parameters

## HTMX Integration

The `Pagination` class automatically configures HTMX attributes for SPA-like navigation:

- **`hx-get`**: Fetches the new page content from the generated route
- **`hx-target`**: Targets the content ID (auto-generated or custom)
- **`hx-swap`**: Uses `outerHTML` to replace the entire content container
- **`hx-push-url`**: Updates browser URL (configurable via `push_url` parameter)
- **`href`**: Provides fallback for non-JavaScript navigation

### Automatic Swap Strategy

The pagination uses `outerHTML` swap by default, which means:
- The route returns a complete `Div` with `id=content_id`
- HTMX replaces the entire element (including the ID)
- This ensures the navigation controls are updated with each page change

### Disabled States

Navigation buttons automatically handle boundary conditions:
- Previous button disabled on page 1 (uses `btn_behaviors.disabled`)
- Next button disabled on last page
- Disabled buttons use DaisyUI styling

### URL Management

When `push_url=True` (default):
- Browser URL updates as users navigate pages
- Users can bookmark specific pages
- Back/forward buttons work correctly
- Preserved query parameters are maintained in URLs

Example URL progression:
```
/library/content?page=1&view=grid&media_type=video
/library/content?page=2&view=grid&media_type=video  # page changed, other params preserved
/library/content?page=3&view=grid&media_type=video
```

### Customization Options

You can customize the pagination behavior:

```python
pagination = Pagination(
    pagination_id="results",
    data_loader=load_results,
    render_items=render_results_list,
    items_per_page=50,
    style=PaginationStyle.COMPACT,  # No page info display
    prev_text="← Previous",  # Custom button text
    next_text="Next →",
    button_size=str(btn_sizes.sm),  # Smaller buttons
    push_url=False  # Don't update URL (for modals, etc.)
)
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()